**Objetivo de este problema es determinar los factores que pueden llevar a los concesionarios a cumplir con los objetivos de satisfacción mínima.**

Datos *deIiveryDatasetChaIlenge.json* incluye datos de 29,389 rutas de camiones, realizadas por 29,389 empleados con las siguientes variables : 

**Primeras 10 columnas**

- **anonId:** Identificador único de la ruta
- **birthdate:** Fecha de nacimiento del empleado
- **routeDate:** Día en que se realizó la ruta entre el 20/05/11 y el 20/05/22
- **region:** Zona del mundo donde se ejecutó la ruta:
    - NA: North America
    - AMESA: Africa, Middle East and South Asia
    - APAC: Asia Pacific, Australia/New Zealand, and China 
    - LATAM: Latin America
    - Europe
- **gender:** Género autodeterminado del empleado {F-Female, M-Male, X-Non binary}
- **areaWealthLevel:** Desarrollo de la zona económica, en comparación con el conjunto del país (Low, Mid or High)
- **areaPopulation:** Población de la zona cubierta, en miles
- **badWeather:** Malas condiciones meteorológicas en la zona, como precipitaciones o viento fuerte
- **weatheRestrictions:** Afectaciones en la zona debido al clima
- **routeTotalDistance:** Distancia de la ruta recorrida en kms

**Segundo set de columnas**

- **numberOfShops:** Total Tiendas que cubrimos en la zona
- **marketShare:** Porcentaje de cuota de mercado que la empresa tiene en la zona en sus categorías.
- **avgAreaBenefits:** Beneficio económico semanal en la zona (en miles de $)
- **timeFromAvg:** Tiempo empleado en la ruta, comparado con la media (negativo significaría que se tardó menos que la media
- **advertising:** Inversión en material de punto de venta en las tiendas (de 0, que significa que no se invierte, a 3, que se invierte mucho)
- **emoloyeeLYScore:** Calificando la puntuación del año pasado (de 1 a 5, siendo 5 la más alta). Los nuevos empleados tienen 3 por defecto.
- **employeeTenure:** Tiempo que el empleado lleva en la empresa
  - 0: se han incorporado en los últimos 12 meses 
  - 1: de 1 a 3 años
  - 2: de 3 a 10 años
  - 3: mas de 1O años
- **emploIoyeePrevComps:** Número de empresas en las que el empleado trabajó anteriormente desarrollando la misma función (5 significa 3 o más).
- **success:** El distribuidor ha distribuido al menos el valor esperado (1) o menos (0)

In [13]:
import warnings
import pandas as pd
import json
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

def edad(birthdate):
    return int(((datetime.now() - birthdate).days)/365.2425)

with open('deliveryDatasetChallenge.json') as json_file:
    data = json.load(json_file)
    
    print("Type:", type(data))


data = pd.DataFrame(data)
data_columns = data.columns.str.split(';',expand=True)

data = data.iloc[:,0].str.split(';',expand=True)

data.columns = ['anonID','birthdate','routeDate','region','gender','areaWealthLevel','areaPopulation',
                'badWeather','weatherRestrictions','routeTotalDistance','numberOfShops','marketShare',
                'avgAreaBenefits','timeFromAvg','advertising','employeeLYScore','employeeTenure','employeePrevComps','success']

data = data.reset_index().drop(["index"], axis =1)

del json_file
del data_columns
data.head()

Type: <class 'dict'>


,anonID,birthdate,routeDate,region,gender,areaWealthLevel,areaPopulation,badWeather,weatherRestrictions,routeTotalDistance,numberOfShops,marketShare,avgAreaBenefits,timeFromAvg,advertising,employeeLYScore,employeeTenure,employeePrevComps,success
0,XLKC1691,23/10/1969,19/05/2020,NA,M,Mid,30.66,No,No,28.3,26,49,18.9,0.35,3,4,1,1,NA
1,KANY6459,14/04/1976,19/05/2020,LATAM,M,Mid,52.5,No,No,21.3,28,96,14.28,0.47,3,4,3,2,NA
2,ULUZ5859,03/04/1985,NA,NA,M,High,25.08,Yes,No,25.9,25,76,14.01,NA,2,3,1,1,NA
3,LDCQ7279,24/03/1968,11/05/2020,NA,M,Mid,37.14,No,No,24.3,26,65,19.2,-0.37,2,4,3,2,NA
4,KIUB5860,22/04/1978,19/05/2020,AMESA,M,Low,11.71,No,No,22.7,22,72,12.37,0.44,2,3,3,2,NA


In [14]:
#####################
#Formato de columnas#
#####################
#Variables númericas : 'areaPopulation','routeTotalDistance','numberOfShops','marketShare', 'avgAreaBenefits', 'timeFromAvg',
#                      'advertising', 'employeeLYScore','employeeTenure', 'employeePrevComps'
data["areaPopulation"] = data["areaPopulation"].astype(float)
data['routeTotalDistance'] = data['routeTotalDistance'].astype(float)
data['numberOfShops'] = data['numberOfShops'].astype(int)
data['marketShare'] = data['marketShare'].astype(float)

data.loc[data['avgAreaBenefits'] == 'NA','avgAreaBenefits'] = 0
data['avgAreaBenefits'] = data['avgAreaBenefits'].astype(float)

data.loc[data['timeFromAvg'] == 'NA','timeFromAvg'] = 0
data['timeFromAvg'] = data['timeFromAvg'].astype(float)

data['advertising'] = data['advertising'].astype(int)
data['employeeLYScore'] = data['employeeLYScore'].astype(int)
data['employeeTenure'] = data['employeeTenure'].astype(int)
data['employeePrevComps'] = data['employeePrevComps'].astype(int)

#Variables categorícas : 'anonID', 'region', 'gender', 'areaWealthLevel','badWeather', 'weatherRestrictions','success'
data.loc[data['success'] == 'NA','success'] = -1
data['success'] = data['success'].astype(int)
data["region-gender-areaWealthLevel-badWeather-weatherRestrictions"] = data["region"] +"-"+ data["gender"] +"-"+ data["areaWealthLevel"] +"-"+ data["badWeather"] +"-"+ data["badWeather"] 

#Variables tipo fecha
data["birthdate"] = pd.to_datetime(data["birthdate"])
data["edad"] = data["birthdate"].apply(edad)

data["routeDate"] = np.where(data["routeDate"] == 'NA', '', data["routeDate"])
data["routeDate"] = pd.to_datetime(data["routeDate"])
data["month"] = data["routeDate"].dt.month
data["day"] = data["routeDate"].dt.day
data["year"] = data["routeDate"].dt.year

#Reacomodo
data = data.loc[:,["anonID","birthdate","edad","routeDate","month","day","year","region","gender","areaWealthLevel","badWeather","weatherRestrictions",
                   "region-gender-areaWealthLevel-badWeather-weatherRestrictions","areaPopulation","routeTotalDistance","numberOfShops","marketShare",
                   "avgAreaBenefits","timeFromAvg","advertising","employeeLYScore","employeeTenure","employeePrevComps","success"]]

data.head()

,anonID,birthdate,edad,routeDate,month,day,year,region,gender,areaWealthLevel,...,routeTotalDistance,numberOfShops,marketShare,avgAreaBenefits,timeFromAvg,advertising,employeeLYScore,employeeTenure,employeePrevComps,success
0,XLKC1691,1969-10-23,52,2020-05-19,5.0,19.0,2020.0,NA,M,Mid,...,28.3,26,49.0,18.90,0.35,3,4,1,1,-1
1,KANY6459,1976-04-14,46,2020-05-19,5.0,19.0,2020.0,LATAM,M,Mid,...,21.3,28,96.0,14.28,0.47,3,4,3,2,-1
2,ULUZ5859,1985-03-04,37,NaT,NaN,NaN,NaN,NA,M,High,...,25.9,25,76.0,14.01,0.00,2,3,1,1,-1
3,LDCQ7279,1968-03-24,54,2020-11-05,11.0,5.0,2020.0,NA,M,Mid,...,24.3,26,65.0,19.20,-0.37,2,4,3,2,-1
4,KIUB5860,1978-04-22,44,2020-05-19,5.0,19.0,2020.0,AMESA,M,Low,...,22.7,22,72.0,12.37,0.44,2,3,3,2,-1
